In [60]:
# Libraries
import pandas as pd

In [61]:
# Load datasets
categories = pd.read_csv("../data/raw/categories.txt")
lower_categories = pd.read_csv("../data/raw/lower_categories.txt")
items_extended = pd.read_csv("../data/raw/items_extended.txt")
items = pd.read_csv("../data/raw/items.txt")

In [62]:
# Remove duplicates
items = items.drop_duplicates(subset="id")
items_extended = items_extended.drop_duplicates(subset="id")

In [63]:
# Show shape of items datasets
print(items.shape) 
print(items_extended.shape)

(1486, 129)
(1486, 92)


In [64]:
# Uniform column naming of items datasets
def uniform_column_names(dataframe):
    new_column_names = [column.replace(".", "_") for column in list(dataframe.columns)]
    dataframe.columns = new_column_names
    return dataframe

items = uniform_column_names(items)
items_extended = uniform_column_names(items_extended)

In [65]:
# Remove columns with high nulls percentage in items datasets
def remove_low_info_columns(dataframe, nulls_percentage_tolerance=0.8):
    valid_columns = list(dataframe.columns)
    rows = dataframe.shape[0]
    
    for column in list(dataframe.columns):
        if column != "original_price":
            nulls = dataframe[column].isnull().sum()
            nulls_percentage = nulls/rows
            if nulls_percentage > nulls_percentage_tolerance: valid_columns.remove(column)
    
    result = dataframe[valid_columns]
    
    return result

items = remove_low_info_columns(items)
items_extended = remove_low_info_columns(items_extended)

In [66]:
# Join items datasets
items_merged = pd.merge(
    items,
    items_extended,
    left_on="id",
    right_on="id",
    suffixes=("","_OVERLAPPING")
)

In [67]:
# Create parent category field in lower categories dataset
def extract_parent(string):
    return "".join([c for c in string[9:18] if c != "'" and c != ","])

lower_categories["parent_category"] = lower_categories.path_from_root. \
    apply(extract_parent)

In [68]:
# Add category name and parent category id
lower_categories = lower_categories[["id", "name", "parent_category"]]

items_merged = pd.merge(
    items_merged,
    lower_categories,
    left_on="category_id",
    right_on="id",
    suffixes=("","_OVERLAPPING")
)

# Rename category name column
items_merged = items_merged.rename(
    columns = {"name": "category_name"}
)

In [69]:
# Add parent category name
items_merged = pd.merge(
    items_merged,
    categories,
    left_on="parent_category",
    right_on="id",
    suffixes=("","_OVERLAPPING")
)

# Rename parent category name column
items_merged = items_merged.rename(
    columns = {"name": "parent_category_name"}
)

In [70]:
# Remove overlapping columns
valid_columns = [column for column in list(items_merged.columns) if not "OVERLAPPING" in column]
items_merged = items_merged[valid_columns]

In [71]:
# Persist preprocessed dataset
items_merged.to_csv("../data/preprocessed/items_merged.txt", index=False)